Name: 02_observed_climatology_postprocessing<br>
Description: Update multidimensional values. This notebook contains the code to update the StdTime dimension in a multidimensional file from a date range to year-end date value. This applies to files in the Esri CRF format.<br>
Date: August 2024<br>
Requirements: Python 3.11.10, xclim 0.47.0, xarray 2023.6.0<br>
Author: Mark Gilbert, Principle GIS Engineer, ArcGIS Living Atlas of the World, Esri (mgilbert@esri.com)<br>

### Python Imports

In [1]:
import arcpy
import xarray as xr
import pandas as pd
import os

### Environment Settings

In [2]:
arcpy.env.overwriteOutput = True

### Input Data Lists

In [ ]:
# Data values for the slices in these multidimensional CRF files are in range format. In order to match the files from NOAA, 
# the ranges need to be converted to a year-end date.
# E.g. '1950-01-01 - 1950-12-31' becomes simply '1950-12-31'.
#
#  TODO: Uncomment the list you want to process.
#  TODO: Replace the path with your own paths.
#
# in_ras_list = [
#     r"replace_with_your_path\Livneh_pr-above-nonzero-99th_1950_2013_annual_inch.crf",
#     r"replace_with_your_path\Livneh_pr-above-nonzero-95th_1950_2013_annual_inch.crf",
#     r"replace_with_your_path\Livneh_pr-above-nonzero-90th_1950_2013_annual_inch.crf",
#     r"replace_with_your_path\Livneh_pr-days-above-nonzero-99th_1950_2013_annual.crf",
#     r"replace_with_your_path\Livneh_pr-days-above-nonzero-95th_1950_2013_annual.crf",
#     r"replace_with_your_path\Livneh_pr-days-above-nonzero-90th_1950_2013_annual.crf",
# ]
# year_range = (1950, 2013) # Livneh

in_ras_list = [
    r"replace_with_your_path\nClimGrid_pr-above-nonzero-99th_1951_2023_annual_inch.crf",
    r"replace_with_your_path\nClimGrid_pr-above-nonzero-95th_1951_2023_annual_inch.crf",
    r"replace_with_your_path\nClimGrid_pr-above-nonzero-90th_1951_2023_annual_inch.crf",
    r"replace_with_your_path\nClimGrid_pr-days-above-nonzero-99th_1951_2023_annual.crf",
    r"replace_with_your_path\nClimGrid_pr-days-above-nonzero-95th_1951_2023_annual.crf",
    r"replace_with_your_path\nClimGrid_pr-days-above-nonzero-90th_1951_2023_annual.crf"
]
year_range = (1951, 2023) # nclimgrid

out_path = r"D:\Data"

### Function Definitions

In [4]:
def get_new_times(range_tuple):
    '''
    Create a reformatted list of datetime values going from a range to a year-end value
    
    inputs:
    range_tuple -- (int) a tuple containing the original start and end year.

    outputs:
    new_time_values -- (str) a string of the output name

    '''
    new_time_values = []
    start_year = range_tuple[0]
    end_year = range_tuple[1]
    for year in range(start_year, end_year + 1):
        new_time_values.append(pd.to_datetime(f"{year}-12-31"))
        
    return new_time_values


def copy_raster(in_ras, out_ras, transpose="NO_TRANSPOSE"):
    '''
    Use the arcpy GP tool to copy the input raster to netCDF format.
    
    inputs:
    in_ras -- (str) The path to the source raster file to be copied.
    out_ras -- (str) Output path of the netCDF copy.

    outputs:
    copy_result[0] -- [str] Path to the output netCDF file.
    
    '''
    with arcpy.EnvManager(rasterStatistics="NONE", snapRaster=in_ras, pyramid="NONE"):
        copy_result = arcpy.management.CopyRaster(
            in_raster=in_ras,
            out_rasterdataset=out_ras,
            pixel_type="32_BIT_FLOAT",
            process_as_multidimensional="ALL_SLICES",
            build_multidimensional_transpose=transpose
        )
        
    return copy_result[0]


def stdtime_to_yearend(in_netcdf, new_time_values):
    '''
    Update the times in StdTime to replace ranges with year-end dates
    '''
    ds = xr.open_dataset(in_netcdf)
    ds["StdTime"] = new_time_values
    
    return ds

In [5]:
def main():
    new_times = get_new_times(year_range)
    for file in in_ras_list:
        out_nc_name = os.path.basename(file).split(".")[0] + "_temp.nc"
        out_nc_path = os.path.join(r"C:\temp", out_nc_name)
        print(f"Copying {file} to netCDF ...")
        temp_nc = copy_raster(file, out_nc_path)

        print("Updating StdTime ...")
        new_time_ras = stdtime_to_yearend(temp_nc, new_times)
        new_nc_name = os.path.basename(file).split(".")[0] + "_new.nc"
        new_nc_path = os.path.join(r"C:\temp", new_nc_name)
        print(f"Writing updated raster to {new_nc_path} ...")
        new_time_ras.to_netcdf(new_nc_path)

        print(f"Copying updated raster to CRF at {file}")
        crf_copy = copy_raster(new_nc_path, file, "TRANSPOSE")

        # print("Cleaning up ...")
        # arcpy.Delete_management(out_nc_path)
        # arcpy.Delete_management(new_nc_path)
    
    print("Processing complete")

In [ ]:
if __name__ == "__main__":
    main()